In [ ]:
!pip install ktrain

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from ktrain import text
import pandas as pd
import numpy as np
import ktrain
import nltk
import re

In [3]:
train = pd.read_csv("train.csv")

In [4]:
print("Shape of train is", train.shape)
train.head()

Shape of train is (104632, 4)


,q_id,eng,class,chapter
0,5eff5e6cbd1a9b18d83ad052,Photodiode is a device\nA. Which is always ope...,12,"SEMICONDUCTOR ELECTRONICS: MATERIALS, DEVICES ..."
1,5efebdc3bd1a9b18d838b84b,When water is heated from \( 0^{\circ} \mathrm...,11,THERMAL PROPERTIES OF MATTER
2,5efdacd9bd1a9b18d835e2c6,Potentiometer measures the potential differenc...,12,CURRENT ELECTRICITY
3,5efdca55bd1a9b18d8363327,"In an isosceles trapezium, the length of one o...",12,APPLICATION OF INTEGRALS
4,5efe9020bd1a9b18d838427d,The normal to the curve \( x^{2}=4 y \)\npassi...,12,APPLICATION OF DERIVATIVES


In [5]:
train = train.iloc[:60000, :]
train = train[['eng', 'class', 'chapter']]
print(train.shape)
train.head()

(60000, 3)


,eng,class,chapter
0,Photodiode is a device\nA. Which is always ope...,12,"SEMICONDUCTOR ELECTRONICS: MATERIALS, DEVICES ..."
1,When water is heated from \( 0^{\circ} \mathrm...,11,THERMAL PROPERTIES OF MATTER
2,Potentiometer measures the potential differenc...,12,CURRENT ELECTRICITY
3,"In an isosceles trapezium, the length of one o...",12,APPLICATION OF INTEGRALS
4,The normal to the curve \( x^{2}=4 y \)\npassi...,12,APPLICATION OF DERIVATIVES


In [6]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import re
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
num_dict = {1: 'one', 2:'two', 3: 'three', 4:'four', 5:'five', 6: 'six', 7:'seven', 8: 'eight', 9: 'nine', 10: 'ten', 11: 'eleven', 12: 'twelve'}

def do_preprocess(df):
    for index, row in df.iterrows():
        filter_sentence = ''
        question = row['eng']
        class_ = row['class']

        sentence = 'class' + num_dict[class_] + ' ' + question


        # Cleaning the sentence with regex
        sentence = re.sub(r'[^\w\s]', '', sentence)

        # Tokenization
        words = nltk.word_tokenize(sentence)

        # Stopwords removal
        words = [w for w in words if not w in stop_words]
        
        for words in words:
            filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()
        
        df.loc[index, 'eng'] = filter_sentence
    df = df[['eng', 'chapter']]
    return(df)

In [8]:
train_preprocessed = do_preprocess(train)

In [9]:
print("Shape:", train_preprocessed.shape)
train_preprocessed.head()

Shape: (60000, 2)


,eng,chapter
0,classtwelve photodiode device a which always ...,"SEMICONDUCTOR ELECTRONICS: MATERIALS, DEVICES ..."
1,classeleven when water heated 0circ mathrmc v...,THERMAL PROPERTIES OF MATTER
2,classtwelve potentiometer measure potential d...,CURRENT ELECTRICITY
3,classtwelve in isosceles trapezium length one...,APPLICATION OF INTEGRALS
4,classtwelve the normal curve x24 passing 12 a...,APPLICATION OF DERIVATIVES


In [10]:
train_preprocessed.chapter.value_counts()

THE p-BLOCK ELEMENTS                  2038
RAY OPTICS AND OPTICAL INSTRUMENTS    1504
EQUILIBRIUM                           1212
STRUCTURE OF ATOM                     1188
MOTION IN A STRAIGHT LINE             1095
                                      ... 
RATIO AND PROPORTION                     6
REACHING THE AGE OF ADOLESCENCE          6
NUTRTION IN ANIMALS                      6
COMPONENTS OF FOOD                       5
INTRODUCTION TO EUCLID’S GEOMETRY        5
Name: chapter, Length: 202, dtype: int64

In [11]:
maxx = -1
for index, row in train_preprocessed.iterrows():
    sentence = row["eng"]
    words = sentence.split()
    maxx = max(maxx, len(words))

print(maxx)

213


In [12]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df= train_preprocessed, 
                                                                      text_column = 'eng',
                                                                      label_columns = 'chapter',
                                                                      maxlen = 256,
                                                                      preprocess_mode = 'bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [13]:
model = text.text_classifier(name= 'bert',
                             train_data= (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 256
done.


In [16]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 18)

In [56]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
3000/3000 [==============================] - 3936s 1s/step - loss: 0.7619 - accuracy: 0.7967 - val_loss: 0.7774 - val_accuracy: 0.7897


In [57]:
predictor = ktrain.get_predictor(learner.model, preproc= preprocess)

In [58]:
val = pd.read_csv('val.csv')

In [59]:
val = do_preprocess(val)

In [60]:
print("Shape", val.shape)
val.head()

Shape (11626, 2)


,eng,chapter
0,classten 70 the length three median triangle ...,TRIANGLES
1,classtwelve an electron accelerates point nea...,ELECTROSTATIC POTENTIAL AND CAPACITANCE
2,classeleven horizontal vertical component for...,SYSTEMS OF PARTICLES AND ROTATIONAL MOTION
3,classtwelve f boldsymbolyboldsymbola co sin 2...,CONTINUITY AND DIFFERENTIABILITY
4,classtwelve 10 a population pt 1000 bacteria ...,APPLICATION OF DERIVATIVES


In [61]:
data = np.array(val['eng'])
y_t = np.array(val['chapter'])

In [62]:
y_pred = predictor.predict(data)
y_pred = np.array(y_pred)

In [63]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print("Accuracy score for test data is:")
print(accuracy_score(y_t, y_pred))

Accuracy score for test data is:
0.7925339755719938


In [64]:
print(classification_report(y_t, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                   precision    recall  f1-score   support

                                           ACIDS, BASES AND SALTS       0.88      0.70      0.78        20
                                     ALCOHOLS, PHENOLS AND ETHERS       0.61      0.60      0.61        86
                          ALDEHYDES, KETONES AND CARBOXYLIC ACIDS       0.54      0.71      0.61       109
                                            ALGEBRAIC EXPRESSIONS       0.50      0.22      0.31         9
                             ALGEBRAIC EXPRESSIONS AND IDENTITIES       0.48      0.74      0.58        19
                                              ALTERNATING CURRENT       0.89      0.88      0.88        99
                                                           AMINES       0.85      0.50      0.63        46
                                      ANATOMY OF FLOWERING PLANTS       0.78      0.91      0.84        43
                                    

In [65]:
def preprocess_sentence(question, class_):
    filter_sentence = ''
    sentence = 'class' + num_dict[class_] + ' ' + question

    # Cleaning the sentence with regex
    sentence = re.sub(r'[^\w\s]', '', sentence)

    # Tokenization
    words = nltk.word_tokenize(sentence)
        
    # Stopwords removal
    words = [w for w in words if not w in stop_words]
        
    for words in words:
        filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()
    
    return(filter_sentence)

In [66]:
def detect(question, class_):
    question = preprocess_sentence(question, class_)
    chapter = predictor.predict(question)
    return('The chapter is ' + chapter)

In [67]:
detect("If  f(x)  is a differentiable function and  g(x) is a double differentiable function such that  |f(x)|≤1  and  f'(x)=g(x) . If  f2(0)+g2(0)=9 . Prove that there exists some  c∈(–3,3) such that g(c).g''(c)<0.", 11)

'The chapter is RELATIONS AND FUNCTIONS'

In [68]:
detect("The pH of a 0.1 M monobasic acid solution is found to be 2. The osmotic pressure of this solution at a temperature of T(K) is", 11)

'The chapter is EQUILIBRIUM'

In [69]:
detect("If the radius of the earth were to shrink by one percent, its mass remaining the same, the acceleration due to gravity on the earth’s surface would", 11)

'The chapter is GRAVITATION'

In [70]:
detect("A wall has two layers A and B, each made of different material. Both the layers have the same thickness. The thermal conductivity of the material of A is twice that of B. Under thermal equilibrium, the temperature difference across the wall is 36° C  The temperature difference across the layer A is", 11)

'The chapter is THERMAL PROPERTIES OF MATTER'

In [71]:
detect("Two bodies M and N of equal masses are suspended from two separate massless springs of spring con­stants k1 and k2 respectively. If the two bodies os­cillate vertically such that their maximum veloci­ties are equal, the ratio of the amplitude of vibra­tion of M to that of N is", 11)

'The chapter is OSCILLATIONS'

In [72]:
detect("Two equal point charges are fixed at x = -a and x = +a on the axis. Another point charge Q is placed at the origin. The change in the electrical potential energy of Q, when it is displaced by a small distance x along the x-axis, is approximately proportional to", 12)

'The chapter is ELECTROSTATIC POTENTIAL AND CAPACITANCE'

In [73]:
detect("A particle of mass m is moving in a circular path of constant radius r such that its centripetal acceleration ac is varying with time ‘t’ as ac = k2rt2 where ‘k’ is a constant. The power delivered to the particle by the force acting on it is", 11)

'The chapter is WORK, ENERGY AND POWER'

In [74]:
detect("A smooth sphere A is moving on a frictionless horizontal plane with angular speed co and centre of mass velocity v. It collides elastically and head on with an identical sphere B at rest. Neglect friction everywhere. After the collision, their angular speeds are (0A and 0)B, respectively. Then", 11)

'The chapter is WORK, ENERGY AND POWER'

In [75]:
detect("if the chord y = mx + 1 of the circle x2 + y2 = 1 subtend an angle of 45 degree as the major segment of the circle, then the value of m is", 11)

'The chapter is CONIC SECTIONS'